# Story Text Regeneration Service Testing - Audio Stories

This notebook tests the story generation API endpoints for **audio stories**:
1. Generate Story Outline (Audio Story)
2. Generate Complete Audio Story
3. Regenerate Story Segment


## Setup and Authentication


In [1]:
import os
import json
import requests
import boto3
from pprint import pprint
from datetime import datetime

# Set AWS environment
os.environ['AWS_PROFILE'] = 'sandbox'
os.environ['AWS_REGION'] = 'us-east-1'

# API Configuration - UPDATE THESE VALUES
API_BASE_URL = 'https://a3aflxx1o2.execute-api.us-east-1.amazonaws.com/dev'  # Update with your API Gateway URL
CLIENT_ID = '1lsusuo902uhpki5tvggo6k16g'  # Update with your Cognito client ID
USERNAME = 'admin@example.com'  # Update with your username
PASSWORD = 'NewPassword123!'  # Update with your password

print("✓ Configuration loaded")


✓ Configuration loaded


In [2]:
# Get authentication token
print("🔐 Getting authentication token...")

cognito_client = boto3.client('cognito-idp', region_name='us-east-1')

auth_response = cognito_client.initiate_auth(
    ClientId=CLIENT_ID,
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME': USERNAME,
        'PASSWORD': PASSWORD
    }
)

TOKEN = auth_response['AuthenticationResult']['IdToken']
print(f"✓ Token obtained ({len(TOKEN)} characters)")

# Prepare headers for API calls
headers = {
    'Authorization': TOKEN,
    'Content-Type': 'application/json'
}

# Initialize S3 client for downloading images
s3_client = boto3.client('s3', region_name='us-east-1')

print("✓ Authentication complete")

🔐 Getting authentication token...
✓ Token obtained (1118 characters)
✓ Authentication complete


## Test 1: Generate Story Outline (Audio Story)

Generate a story outline for an **audio story** about a brave mouse named Max.


In [3]:
print("📋 TEST 1: Generate Story Outline (Audio Story)")
print("─" * 70)

# Generate unique job ID
job_id_base = f"complete-workflow-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
outline_job_id = f"{job_id_base}-outline"

outline_payload = {
    "genre": "adventure",
    "reading_level": "elementary",
    "tone": "lighthearted",
    "user_input_description": "A brave little mouse named Max who discovers a magical cheese that grants wishes",
    "story_type": "audio",
    "number_of_speakers": 3,
    "audio_length": 120,  # 5 minutes in seconds
    "job_id": outline_job_id,
    "model_id": "bedrock/openai.gpt-oss-120b-1:0"
}

print("\nRequest payload:")
pprint(outline_payload)

print("\nSending request...")
response = requests.post(
    f"{API_BASE_URL}/generate-story-outline",
    headers=headers,
    json=outline_payload
)

print(f"\nResponse Status: {response.status_code}")
print("\nResponse Body:")
outline_response = response.json()
pprint(outline_response)

if response.status_code == 200:
    print("\n✅ PASS - Audio story outline generation successful")
    story_parts = outline_response['result']['story_parts']
    print("\n📝 Story parts extracted for next test")
    
    # Export response to JSON file
    output_file = "test1_audio_outline_response.json"
    with open(output_file, 'w') as f:
        json.dump(outline_response, f, indent=2)
    print(f"💾 Response exported to: {output_file}")
else:
    print(f"\n❌ FAIL - Expected 200 but got {response.status_code}")
    raise Exception("Test 1 failed")


📋 TEST 1: Generate Story Outline (Audio Story)
──────────────────────────────────────────────────────────────────────

Request payload:
{'audio_length': 120,
 'genre': 'adventure',
 'job_id': 'complete-workflow-20251014-121730-outline',
 'model_id': 'bedrock/openai.gpt-oss-120b-1:0',
 'number_of_speakers': 3,
 'reading_level': 'elementary',
 'story_type': 'audio',
 'tone': 'lighthearted',
 'user_input_description': 'A brave little mouse named Max who discovers a '
                           'magical cheese that grants wishes'}

Sending request...

Response Status: 200

Response Body:
{'authenticated_user': {'email': 'admin@example.com',
                        'sub': '24e8e4f8-0061-70c1-0b17-2508a1b5b9a6',
                        'username': '24e8e4f8-0061-70c1-0b17-2508a1b5b9a6'},
 'request_id': '24cdeaf6-6541-4923-a2ac-16f7e75435ba',
 'result': {'metadata': {'audio_length': 120,
                         'created_timestamp': '2025-10-14T16:17:59.714746+00:00',
                        

## Test 2: Generate Complete Audio Story

Generate a complete **audio story** using the outline from Test 1.


In [4]:
print("📋 TEST 2: Generate Complete Story (Audio, 5 minutes)")
print("─" * 70)

story_job_id = f"{job_id_base}-story"

story_payload = {
    "genre": "adventure",
    "reading_level": "elementary",
    "tone": "lighthearted",
    "story_outline_description": story_parts,
    "story_type": "audio",
    "number_of_speakers": 3,
    "audio_length": 120,  # 5 minutes in seconds
    "job_id": story_job_id,
    "model_id": "bedrock/openai.gpt-oss-120b-1:0"
}

print("\nRequest payload (story_outline_description from previous test):")
pprint(story_payload)

print("\nSending request...")
response = requests.post(
    f"{API_BASE_URL}/generate-story",
    headers=headers,
    json=story_payload
)

print(f"\nResponse Status: {response.status_code}")
print("\nResponse Body:")
story_response = response.json()
pprint(story_response)

if response.status_code == 200:
    print("\n✅ PASS - Complete audio story generation successful")
    
    # Extract segments for regeneration test
    segments = []
    for story_part in story_response['result']['story_parts']:
        for section in story_part['sections']:
            for segment in section['segments']:
                segments.append({
                    'segment_num': segment['segment_num'],
                    'segment_content': segment['segment_content'],
                    'speaker': segment['speaker']
                })
    
    # Pick the 3rd segment for regeneration (index 2)
    segment_to_regen = segments[2]
    segment_num = segment_to_regen['segment_num']
    
    print(f"\n📝 Extracted segment #{segment_num} for regeneration test")
    print(f"\n📊 Audio Story Structure:")
    print(f"  • Total segments: {len(segments)}")
    print(f"  • Story parts: {len(story_response['result']['story_parts'])}")
    
    # Show story structure
    for i, part in enumerate(story_response['result']['story_parts'], 1):
        print(f"  • Part {i} ({part['story_part']}): {len(part['sections'])} sections")
        for j, section in enumerate(part['sections'], 1):
            print(f"    - Section {j}: {len(section['segments'])} segments")
    
    # Export response to JSON file
    output_file = "test2_audio_story_response.json"
    with open(output_file, 'w') as f:
        json.dump(story_response, f, indent=2)
    print(f"\n💾 Response exported to: {output_file}")
else:
    print(f"\n❌ FAIL - Expected 200 but got {response.status_code}")
    raise Exception("Test 2 failed")


📋 TEST 2: Generate Complete Story (Audio, 5 minutes)
──────────────────────────────────────────────────────────────────────

Request payload (story_outline_description from previous test):
{'audio_length': 120,
 'genre': 'adventure',
 'job_id': 'complete-workflow-20251014-121730-story',
 'model_id': 'bedrock/openai.gpt-oss-120b-1:0',
 'number_of_speakers': 3,
 'reading_level': 'elementary',
 'story_outline_description': [{'story_part': 'beginning',
                                'story_part_speakers': ['Narrator',
                                                        'Max the Mouse',
                                                        'Cheesy the Cheese'],
                                'story_part_summary': 'In the sunny meadow '
                                                      "just beyond Max's cozy "
                                                      'burrow, the brave '
                                                      'little mouse spots '
                    

a